# Task 2 - Konsep Sistem Rekomendasi Pada Ecommerce

Mempelajari & Memahami konsep sistem rekomendasi pada ecommerce dengan metode colaborative filtering lalu membuat aplikasi sederhana dengan konsep tersebut beserta penjelasan, penerapan / implementasinya.

Metode Collaborative Filtering merupakan salah satu metode pada sistem rekomendasi yang memanfaatkan penilaian pengguna (user) lain berupa rating atau umpan balik lain untuk memprediksi item yang mungkin diminati.

# Import Library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
 
sns.set_style('white')
%matplotlib inline

# Read Data

Pada tahap ini saya menggunakan 2 data yg dimana kedua data ini berasal dari Web www.geeksforgeeks.org. Isi dari data ini adalah mengenai Film beserta Ratingnya

In [2]:
df_id = pd.read_csv('file.tsv', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
df_id.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [4]:
df_movies = pd.read_csv('Movie_Id_Titles.csv')
df_movies.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


# Data Preprocessing

Kedua data yg barysan dibaca, akan digabungkan menjadi 1 dengan nama df_merged

In [6]:
df_merged = pd.merge(df_id, df_movies, on='item_id')
df_merged.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [7]:
# df_merged.groupby('title')['rating'].mean().sort_values(ascending = False).head()

In [8]:
# df_merged.groupby('title')['rating'].count().sort_values(ascending=False).head()

Membuat Dataframe dengan rating count values

In [9]:
ratings = pd.DataFrame(df_merged.groupby('title')['rating'].mean())
ratings['num of ratings'] = pd.DataFrame(df_merged.groupby('title')['rating'].count())
ratings.head()

,rating,num of ratings
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41


Membuat Matriks Film, Komponen utama dari sistem rekomendasi adalah matriks. Ini menyatakan rating setiap Film yang ada pada Data. Untuk membuat Matriks dapat menggunakan fungsi pivot_table(). User_id akan ditetapkan sebagai kolom indeks, dan Judul-judl film akan ditempatkankan dikolom, dengan jumlah rating yang disimpan di setiap sel. Nilai NaN apa pun akan diganti dengan nol.

In [10]:
moviemat = df_merged.pivot_table(index = 'user_id', columns = 'title', values = 'rating').fillna(0)
moviemat.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Recomendation

Saat menjalankan fungsi get_recommendations() akan meneruskan kerangka data matriks item yang berisi setiap film dan rating. Fungsi pemberi rekomendasi kemudian akan menghitung korelasi Pearson untuk Film tersebut dan mengembalikan Film yang paling berkorelasi menggunakan fungsi corrwith(), sehingga menghasilkan Film yang akurat untuk ditampilkan. Fungsi tersebut akan mengembalikan rekomendasi Film berdasarkan Film yang paling sering dikaitkan dengan item tersebut.

In [11]:
def get_recomendations(df_merged, item):
    recomendations = moviemat.corrwith(moviemat[item])
    recomendations = pd.DataFrame(recomendations, columns =['Correlation'])
    recomendations.dropna(inplace = True)
    recomendations = recomendations.join(ratings['num of ratings'])
    recomendations = recomendations[recomendations['num of ratings']>100].sort_values('Correlation', ascending = False)
    
    return recomendations

In [12]:
name_item = 'Return of the Jedi (1983)'
recomendations = get_recomendations(df_merged, name_item)
recomendations.head()

,Correlation,num of ratings
title,,
Return of the Jedi (1983),1.000000,507
Star Wars (1977),0.746797,584
"Empire Strikes Back, The (1980)",0.548797,368
Raiders of the Lost Ark (1981),0.503323,420
Indiana Jones and the Last Crusade (1989),0.478037,331


# Flask Web

In [ ]:
import numpy as np
import pickle
from flask import Flask, request, render_template

app = Flask(__name__, template_folder = 'template' )


@app.route('/')
def home():
    return render_template('Movie_Recommendation.html')

@app.route('/predict', methods = ['POST'])
def predict():
    #result  = 
    recomendations = moviemat.corrwith(moviemat[request.form["item"]])
    recomendations = pd.DataFrame(recomendations, columns =['Correlation'])
    recomendations.dropna(inplace = True)
    recomendations = recomendations.join(ratings['num of ratings'])
    recomendations = recomendations[recomendations['num of ratings']>100].sort_values('Correlation', ascending = False).head()
    result = recomendations
    
    return render_template('Movie_Recommendation.html', tables=[result.to_html(classes='data')], titles=result.columns.values)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Sep/2021 14:12:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 14:12:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Sep/2021 14:12:04] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 14:12:12] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 14:31:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 14:31:23] "POST /predict HTTP/1.1" 200 -
